# ProperImage con DES Cluster Simulation

Analizaremos la PSF de las simulaciones de **DES**.
Para ello utilizaremos a properimage para analizar cada imagen y calcular la PSF variable en el espacio.
Primero obteniendo las *autopsfs*, luego calculando la PSF estimada en una grilla en el espacio imagen, 
y por ultimo en la posicion individual de cada galaxia.

Posteriormente ajustaremos una gaussiana 2d a cada posicion de la grilla para calcular 
las propiedades de la psf, y finalmente estimar cantidades como elipticidad.

In [1]:
import os
from glob import glob 

import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

#%matplotlib inline

from properimage import single_image as si
from properimage import utils
from properimage import plot

In [2]:
os.chdir('/home/bruno/Devel/DESCSimulation/DES-sextractor/')

In [3]:
#librerias propias
import psf_DES
import test_psf_DES

In [4]:
from astropy.convolution import convolve

In [5]:
from astropy.io import ascii
from astropy.table import Table, Row

In [6]:
from astropy.nddata.utils import extract_array

In [7]:
image_path = os.path.abspath('./../imagenes-LN-PSF')

In [8]:
images = glob(pathname=image_path+'/*.fits')

In [9]:
from astropy.modeling import models, fitting
def get_single_psf(x, y, a_fields, psf_basis):
    s = np.zeros_like(psf_basis[0])
    
    for i in range(len(psf_basis)):
        s += psf_basis[i]*a_fields[i](x,y)
    
    x, y = np.mgrid[:s.shape[0], :s.shape[1]]
    g2 = models.Gaussian2D(x_stddev=1, y_stddev=1, 
                           x_mean=s.shape[0]/2, 
                           y_mean=s.shape[1]/2)
    g2 = g2 + models.Const2D()
    fitter = fitting.LevMarLSQFitter()
    
    return fitter(g2, x, y, s)[0]

In [10]:
def write_im2shape_psftab(table, filepath):
    with open(filepath, 'w') as f:
        f.write('1\n')
        f.write(str(len(table)) + '\n')
        f.write('6\n')
        table.write(f, format='ascii.csv')

In [11]:
def write_im2shape_gxstab(table, filepath):
    with open(filepath, 'w') as f:
        f.write('26 '+str(len(table)) + '\n')
        table.write(f, format='ascii.csv')

In [12]:
seeing_par = {}
seeing_par['pixsize'] = 0.27
seeing_par['run'] = 33
seeing_par['filter'] = 'R'
seeing_par['magmax'] = 23.
seeing_par['magmin'] = 18.
seeing_par['fwhmmax'] = 4.5
seeing_par['plot'] = None
fitter = fitting.LevMarLSQFitter()
for jj in range(len(images)):
    seeing_par['img'] = images[jj]

    psf_grid_table = seeing_par['img'].strip('.fits') + '_psf_grid.csv'
    gxs_table = seeing_par['img'].strip('.fits') + '_gxs.csv'
    psf_gxs_table = seeing_par['img'].strip('.fits') + '_psf_gxs.csv'
    psf_stars_table_model = seeing_par['img'].strip('.fits') + '_psf_stars_model.csv'
    psf_stars_table_data = seeing_par['img'].strip('.fits') + '_psf_stars_data.csv'
    im2shape_psf_tab = seeing_par['img'].strip('.fits') + '_i2s_psf_tab.csv'
    im2shape_gxs_tab = seeing_par['img'].strip('.fits') + '_i2s_gxs_tab.csv'
    
    af_path = seeing_par['img'].strip('.fits') + '_a_fields.png'
    pb_path = seeing_par['img'].strip('.fits') + '_psf_basis.png'

    with psf_DES.SingleImageDES(seeing_par=seeing_par, img=images[jj]) as image:
        a_fields, psf_basis = image.get_variable_psf(inf_loss=0.0005)
        gxs = image.gxs
        srcs = image.best_sources
        lines = []
        for asrc in srcs:
            x = asrc['x']
            y = asrc['y']

            p = get_single_psf(x, y, a_fields, psf_basis)
            sx = p.x_stddev.value
            sy = p.y_stddev.value
            th = ((p.theta.value/np.pi) % 2.)*np.pi  # is in radians
            theta=np.rad2deg(th) # in degrees

            a = 2*max([sx, sy])
            b = 2*min([sx, sy])

            ellip = (a - b)/float(a + b)
            C=np.zeros((2,2),float)
            C[0,0]=2*(((np.cos(th))**2)/a**2+((np.sin(th))**2)/b**2)
            C[1,1]=2*((((np.cos(th))**2)/b**2)+(((np.sin(th))**2)/a**2))
            C[0,1]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            C[1,0]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            A=2*(2*np.pi*np.linalg.det(C))

            lines.append([x, y, sx, sy, ellip, th, a*b, a, b, C[0,0], C[0,1], C[1,0], C[1,1], A])
        names = ('x', 'y', 'sx', 'sy', 'ellip', 'theta', 'a_times_b', 
                 'a', 'b', 'c11', 'c12', 'c21', 'c22', 'detc')
        PSF_onstars = Table(rows=lines, names=names)
        PSF_onstars.write(psf_stars_table_model, format='ascii.csv', overwrite=True)
        lines = []
        for i in range(image.n_sources):
            asrc = srcs[i]
            x = asrc['x']
            y = asrc['y']
            
            psfi_render = image.db.load(i)[0]
            psx, psy = psfi_render.shape
            g2 = models.Gaussian2D(x_stddev=1, y_stddev=1, 
                           x_mean=psx/2., 
                           y_mean=psy/2.)
            g2 = g2 + models.Const2D()
            fx, fy = np.mgrid[:psx, :psy]
            fit = fitter(g2, fx, fy, psfi_render)
            p = fit[0]
            sx = p.x_stddev.value
            sy = p.y_stddev.value
            th = ((p.theta.value/np.pi) % 2.)*np.pi  # is in radians
            theta=np.rad2deg(th) # in degrees

            a = 2*max([sx, sy])
            b = 2*min([sx, sy])

            ellip = (a - b)/float(a + b)
            C=np.zeros((2,2),float)
            C[0,0]=2*(((np.cos(th))**2)/a**2+((np.sin(th))**2)/b**2)
            C[1,1]=2*((((np.cos(th))**2)/b**2)+(((np.sin(th))**2)/a**2))
            C[0,1]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            C[1,0]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            A=2*(2*np.pi*np.linalg.det(C))

            lines.append([x, y, sx, sy, ellip, th, a*b, a, b, C[0,0], C[0,1], C[1,0], C[1,1], A])
        names = ('x', 'y', 'sx', 'sy', 'ellip', 'theta', 'a_times_b', 
                 'a', 'b', 'c11', 'c12', 'c21', 'c22', 'detc')
        PSF_onstars = Table(rows=lines, names=names)
        PSF_onstars.write(psf_stars_table_data, format='ascii.csv', overwrite=True)
    
    x, y = image.get_afield_domain()
    plot.plot_afields(a_fields, x, y, nbook=False, path=af_path)
    plot.plot_psfbasis(psf_basis=psf_basis, nbook=False, path=pb_path)
    
    shy, shx = image.pixeldata.shape
    lines = []
    for x in np.arange(32, shx, 64):
        for y in np.arange(32, shy, 64):
            p = get_single_psf(x, y, a_fields, psf_basis)
            sx = p.x_stddev.value
            sy = p.y_stddev.value
            th = ((p.theta.value/np.pi) % 2.)*np.pi  # is in radians
            theta=np.rad2deg(th) # in degrees

            a = 2*max([sx, sy])
            b = 2*min([sx, sy])

            ellip = (a - b)/float(a + b)
            C=np.zeros((2,2),float)
            C[0,0]=2*(((np.cos(th))**2)/a**2+((np.sin(th))**2)/b**2)
            C[1,1]=2*((((np.cos(th))**2)/b**2)+(((np.sin(th))**2)/a**2))
            C[0,1]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            C[1,0]=((1/(b**2))-(1/(a**2)))*np.sin(2*th)
            A=2*(2*np.pi*np.linalg.det(C))

            lines.append([x, y, sx, sy, ellip, th, a*b, a, b, C[0,0], C[0,1], C[1,0], C[1,1], A])
    names = ('x', 'y', 'sx', 'sy', 'ellip', 'theta', 'a_times_b', 
             'a', 'b', 'c11', 'c12', 'c21', 'c22', 'detc')
    psf_grid = Table(rows=lines, names=names)
    psf_grid.write(psf_grid_table, format='ascii.csv', overwrite=True)

    #lines = []
    #for row in gxs:
    #    p = get_single_psf(row['X_IMAGE'], row['Y_IMAGE'], a_fields, psf_basis)
    #    sx = p.x_stddev
    #    sy = p.y_stddev
    #    th = p.theta

    #    a = 2*max([sx, sy])
    #    b = 2*min([sx, sy])

    #    ellip = (a - b)/float(a + b)

    #    lines.append([row['X_IMAGE'], row['Y_IMAGE'], 0., 0., ellip, th-np.pi/4., a*b, 1., a, b])


    #PSF_ongxs = Table(rows=lines, 
    #                  names=('x', 'y', 'zero1', 'zero2', 'ellip', 'theta', 'a_times_b', 'one', 'a', 'b'))
    #PSF_ongxs.write(psf_gxs_table, format='ascii.csv', overwrite=True)

    #write_im2shape_psftab(table=gxs, filepath=im2shape_psf_tab)
    #write_im2shape_gxstab(table=gxs, filepath=im2shape_gxs_tab)

    #gxs.write(gxs_table, format='ascii.csv', overwrite=True)
    

 
 ------------ seeing:  0.81945
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_35.fits
 
cantidad de estrellas seleccionadas:  241
cantidad de galaxias seleccionadas:  1336
Sources good to calculate = 241
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 56210)


cleaning... 
 
 ------------ seeing:  0.83295
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_47.fits
 
cantidad de estrellas seleccionadas:  223
cantidad de galaxias seleccionadas:  1744
Sources good to calculate = 223
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 55741)
cleaning... 
 
 ------------ seeing:  0.84105
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_6.fits
 
cantidad de estrellas seleccionadas:  223
cantidad de galaxias seleccionadas:  1568
Sources good to calculate = 223
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 25503)
cleaning... 
 
 ------------ seeing:  0.83835
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_61.fits
 
cantidad de estrellas seleccionadas:  251
cantidad de galaxias seleccionadas:  1359
Sources good to calculate = 251
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 25159)
cleaning... 
 
 ------------ seeing:  

('Masked pixels: ', 70838)
cleaning... 
 
 ------------ seeing:  0.83295
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_15.fits
 
cantidad de estrellas seleccionadas:  247
cantidad de galaxias seleccionadas:  1301
Sources good to calculate = 247
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 33493)
cleaning... 
 
 ------------ seeing:  0.83565
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_38.fits
 
cantidad de estrellas seleccionadas:  253
cantidad de galaxias seleccionadas:  1576
Sources good to calculate = 253
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 32280)
cleaning... 
 
 ------------ seeing:  0.82755
 ------------ imagen:  /home/bruno/Devel/DESCSimulation/imagenes-LN-PSF/im_06_3.fits
 
cantidad de estrellas seleccionadas:  234
cantidad de galaxias seleccionadas:  1606
Sources good to calculate = 234
returning best sources
stamps will be 9 x 9
('Masked pixels: ', 28397)
cleaning...

OSError: [Errno 12] Cannot allocate memory

In [ ]:
for jj in range(len(images)):
    seeing_par['img'] = images[jj]

    psf_grid_table = seeing_par['img'].strip('.fits') + '_psf_grid.csv'
    gxs_table = seeing_par['img'].strip('.fits') + '_gxs.csv'
    psf_gxs_table = seeing_par['img'].strip('.fits') + '_psf_gxs.csv'
    
    psf_table = ascii.read(psf_grid_table)

    x = psf_table['x']
    y = psf_table['y']
    e = psf_table['ellip'] % 2.*np.pi
    a = psf_table['a']#1./(1.-psf_table['ellip'])
    u = a * np.cos(psf_table['theta'])
    v = a * np.sin(psf_table['theta'])
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=e, cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_ellip_scatter.png')
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=psf_table['detc'], cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_detc_scatter.png')
    
    f = np.arange(0, len(x), 5)
    plt.figure(figsize=(12, 12))
    plt.quiver(x[f], y[f], u[f], v[f], psf_table['a_times_b'][f])
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_angles_areas.png')
    
    ################
    psf_stars_table = seeing_par['img'].strip('.fits') + '_psf_stars_data.csv'
    
    psf_table = ascii.read(psf_stars_table)

    x = psf_table['x']
    y = psf_table['y']
    e = psf_table['ellip'] % 2.*np.pi
    a = psf_table['a']#1./(1.-psf_table['ellip'])
    u = a * np.cos(psf_table['theta'])
    v = a * np.sin(psf_table['theta'])
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=e, cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_ellip_scatter_stars_data.png')
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=psf_table['detc'], cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_detc_scatter_stars_data.png')
    
    f = np.arange(0, len(x), 5)
    plt.figure(figsize=(12, 12))
    plt.quiver(x[f], y[f], u[f], v[f], psf_table['a_times_b'][f])
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_angles_areas_stars_data.png')
    
    ################
    psf_stars_table = seeing_par['img'].strip('.fits') + '_psf_stars_model.csv'
    
    psf_table = ascii.read(psf_stars_table)

    x = psf_table['x']
    y = psf_table['y']
    e = psf_table['ellip'] % 2.*np.pi
    a = psf_table['a']#1./(1.-psf_table['ellip'])
    u = a * np.cos(psf_table['theta'])
    v = a * np.sin(psf_table['theta'])
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=e, cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_ellip_scatter_stars_model.png')
    
    plt.figure(figsize=(12, 12))
    plt.scatter(x, y, c=psf_table['detc'], cmap='viridis')
    plt.colorbar()
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_detc_scatter_stars_model.png')
    
    f = np.arange(0, len(x), 5)
    plt.figure(figsize=(12, 12))
    plt.quiver(x[f], y[f], u[f], v[f], psf_table['a_times_b'][f])
    plt.axes().set_aspect('equal')
    plt.savefig(seeing_par['img'].strip('.fits') + '_angles_areas_stars_model.png')
    